# 2 way ANOVA

In [ ]:
import pandas as pd
from statsmodels.graphics.factorplots import interaction_plot
import matplotlib.pyplot as plt
from scipy import stats

Read file into pandas dataframe:

In [ ]:
datafile="/home/vital/Desktop/2wayANOVA/Quanti_PeptMatrix.csv"
data = pd.read_csv(datafile)

Calculate Degrees of Freedom:

In [ ]:
def dof(peptide_df):
	N = len(peptide_df.intensity)
	dof_cell_line = len(peptide_df.cell_line.unique()) - 1
	dof_drug = len(peptide_df.drug.unique()) - 1
	dof_b = dof_cell_line*dof_drug #between
	dof_w = N - (len(peptide_df.cell_line.unique())*len(peptide_df.drug.unique())) #within
	return(dof_cell_line, dof_drug, dof_b, dof_w)


Calculate Sum of Squares:

In [ ]:
def ssq(data, grand_mean):
	#SUM of SQUARES FOR EACH FACTOR (Independent variables: cell_line AND drug): 
	ssq_cell_line = sum( [(data[data.cell_line == l].intensity.mean() - grand_mean)**2 for l in data.cell_line] )
	#Note: Do I need to iterate in data.cell_line or just data.cell_line
	ssq_drug = sum( [(data[data.drug == d].intensity.mean() - grand_mean)**2 for d in data.drug] )
	#TOTAL SUM OF SQUARES:
	ssq_t = sum((data.intensity - grand_mean)**2)
	#SUM OF SQUARES WITHIN (Also called residual)
	cto = data[data.drug == 'CT0']
	fu = data[data.drug == '5-FU']
	doxo = data[data.drug == 'DOXO']
	mtx = data[data.drug == 'MTX']
	ct72 = data[data.drug == 'CT72']
	pctl = data[data.drug == 'PCTL']
	sen = data[data.drug == 'SEN']
	tdx = data[data.drug == 'TDX']
	#
	cto_cell_line_means = [cto[cto.cell_line == c].intensity.mean() for c in cto.cell_line]
	fu_cell_line_means = [fu[fu.cell_line == c].intensity.mean() for c in fu.cell_line]
	doxo_cell_line_means = [doxo[doxo.cell_line == c].intensity.mean() for c in doxo.cell_line]
	mtx_cell_line_means = [mtx[mtx.cell_line == c].intensity.mean() for c in mtx.cell_line]
	ct72_cell_line_means = [ct72[ct72.cell_line == c].intensity.mean() for c in ct72.cell_line]
	pctl_cell_line_means = [pctl[pctl.cell_line == c].intensity.mean() for c in pctl.cell_line]
	sen_cell_line_means = [sen[sen.cell_line == c].intensity.mean() for c in sen.cell_line]
	tdx_cell_line_means = [tdx[tdx.cell_line == c].intensity.mean() for c in tdx.cell_line]
	#
	ssq_w = sum((cto.intensity - cto_cell_line_means)**2) + \
					sum((fu.intensity - fu_cell_line_means)**2) + \
					sum((doxo.intensity - doxo_cell_line_means)**2) + \
					sum((mtx.intensity - mtx_cell_line_means)**2) + \
					sum((ct72.intensity - ct72_cell_line_means)**2) + \
					sum((pctl.intensity - pctl_cell_line_means)**2) + \
					sum((sen.intensity - sen_cell_line_means)**2) + \
					sum((tdx.intensity - tdx_cell_line_means)**2) 
	#SUM OF SQUARES OF THE INTERACTION (BETWEEN)
	ssq_b = ssq_t-ssq_cell_line-ssq_drug-ssq_w
	return(ssq_cell_line, ssq_drug, ssq_t, ssq_w, ssq_b)


Calculate Mean Squares:

In [ ]:
def msq(ssq_cell_line, ssq_drug, ssq_b, ssq_w, dof_cell_line, dof_drug, dof_b, dof_w):
	msq_cell_line = ssq_cell_line / dof_cell_line
	msq_drug = ssq_drug / dof_drug
	msq_b = ssq_b / dof_b #BETWEEN
	msq_w = ssq_w/dof_w #WITHIN
	return(msq_cell_line, msq_drug, msq_b, msq_w)


Calculate F-ratio: 
The F-statistic is simply the mean square for each effect and the interaction divided by the mean square for within (error/residual).

In [ ]:
def f_ratio(msq_cell_line, msq_drug, msq_b, msq_w):
	f_cell_line = msq_cell_line/msq_w
	f_drug = msq_drug/msq_w
	f_b = msq_b/msq_w
	return(f_cell_line, f_drug, f_b)


Create pandas sub-dataframe for each peptide (row in original dataframe)
with columns intensity, cell_line, and drug 

In [ ]:
for index, row in data.iterrows():
	#Get pandas df for each peptide with columns intensity, cell_line and drug:
	print index
	peptide_df = pd.DataFrame(columns=["intensity", "cell_line", "drug"])
	peptide_df.cell_line = ["Melanoma"] * 24 + ["Lung"] * 24 + ["Colon"] * 24
	peptide_df.drug = (["CTO"]*3 + ["5-FU"]*3 + ["DOXO"]*3 + ["MTX"]*3 + ["CT72"]*3 + ["PCTL"]*3 + ["SEN"]*3 + ["TDX"]*3)*3
	peptide_df.intensity = row.values[1:-1]
	#get Degrees of Freedom
	dof_cell_line, dof_drug, dof_b, dof_w = dof(peptide_df)
	#get grand mean Using Pandas DataFrame method mean on the dependent variable only
	grand_mean = peptide_df.intensity.mean() #which is simply the mean of all intensity values
	#Calculate sum of squares
	ssq_cell_line, ssq_drug, ssq_t, ssq_w, ssq_b = ssq(peptide_df, grand_mean)
	#Calculate mean squares
	msq_cell_line, msq_drug, msq_b, msq_w = msq(ssq_cell_line, ssq_drug, ssq_b, ssq_w, dof_cell_line, dof_drug, dof_b, dof_w)
	#Calculate F-ratio (F-satistic)
	f_cell_line, f_drug, f_b = f_ratio(msq_cell_line, msq_drug, msq_b, msq_w)
	#OBTAIN p-values
	#scipy.stats method f.sf checks if the obtained F-ratios are above the critical value. 
	#Use F-value for each effect and interaction as well as the degrees of freedom for them, and the degree of freedom within.
	#Null Hypothesis 1:  H0: cell-lines having the same response
	p_cell_line = stats.f.sf(f_cell_line, dof_cell_line, dof_w)
	#Null Hypothesis 2:  H0: drugs having the same response
	p_drug = stats.f.sf(f_drug, dof_drug, dof_w)
	#Null Hypothesis 3:  H0: cell-lines do not interact with drugs in the response
	p_cell_line_x_drug = stats.f.sf(f_b, dof_b, dof_w)
	#ADD rows to the original pandas df with the three pvalues:
	data.loc[index, 'p_cell_line'] = p_cell_line
	data.loc[index, 'p_drug'] = p_drug
	data.loc[index, 'p_cell_line_x_drug'] = p_cell_line_x_drug
